# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f285008f070>
├── 'a' --> tensor([[ 0.4685,  0.0923,  1.8565],
│                   [ 0.4468,  1.4120, -1.1325]])
└── 'x' --> <FastTreeValue 0x7f285008f580>
    └── 'c' --> tensor([[ 0.3901, -1.8304,  1.2115,  0.6625],
                        [-0.3824, -0.3697, -1.0808, -1.1305],
                        [ 1.1624, -0.4730,  0.5559, -0.4295]])

In [4]:
t.a

tensor([[ 0.4685,  0.0923,  1.8565],
        [ 0.4468,  1.4120, -1.1325]])

In [5]:
%timeit t.a

63.9 ns ± 0.0523 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f285008f070>
├── 'a' --> tensor([[-0.8954, -0.8344, -0.7782],
│                   [-0.3011, -0.5655,  0.4747]])
└── 'x' --> <FastTreeValue 0x7f285008f580>
    └── 'c' --> tensor([[ 0.3901, -1.8304,  1.2115,  0.6625],
                        [-0.3824, -0.3697, -1.0808, -1.1305],
                        [ 1.1624, -0.4730,  0.5559, -0.4295]])

In [7]:
%timeit t.a = new_value

67.2 ns ± 0.0774 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.4685,  0.0923,  1.8565],
               [ 0.4468,  1.4120, -1.1325]]),
    x: Batch(
           c: tensor([[ 0.3901, -1.8304,  1.2115,  0.6625],
                      [-0.3824, -0.3697, -1.0808, -1.1305],
                      [ 1.1624, -0.4730,  0.5559, -0.4295]]),
       ),
)

In [10]:
b.a

tensor([[ 0.4685,  0.0923,  1.8565],
        [ 0.4468,  1.4120, -1.1325]])

In [11]:
%timeit b.a

59.1 ns ± 0.0197 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.2883,  2.1595, -0.8078],
               [-1.5392, -1.3806, -1.9964]]),
    x: Batch(
           c: tensor([[ 0.3901, -1.8304,  1.2115,  0.6625],
                      [-0.3824, -0.3697, -1.0808, -1.1305],
                      [ 1.1624, -0.4730,  0.5559, -0.4295]]),
       ),
)

In [13]:
%timeit b.a = new_value

491 ns ± 0.0635 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

890 ns ± 1.81 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.8 µs ± 12.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

143 µs ± 1.46 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 366 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f27a2cb38b0>
├── 'a' --> tensor([[[ 0.4685,  0.0923,  1.8565],
│                    [ 0.4468,  1.4120, -1.1325]],
│           
│                   [[ 0.4685,  0.0923,  1.8565],
│                    [ 0.4468,  1.4120, -1.1325]],
│           
│                   [[ 0.4685,  0.0923,  1.8565],
│                    [ 0.4468,  1.4120, -1.1325]],
│           
│                   [[ 0.4685,  0.0923,  1.8565],
│                    [ 0.4468,  1.4120, -1.1325]],
│           
│                   [[ 0.4685,  0.0923,  1.8565],
│                    [ 0.4468,  1.4120, -1.1325]],
│           
│                   [[ 0.4685,  0.0923,  1.8565],
│                    [ 0.4468,  1.4120, -1.1325]],
│           
│                   [[ 0.4685,  0.0923,  1.8565],
│                    [ 0.4468,  1.4120, -1.1325]],
│           
│                   [[ 0.4685,  0.0923,  1.8565],
│                    [ 0.4468,  1.4120, -1.1325]]])
└── 'x' --> <FastTreeValue 0x7f27a2cb3580>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.6 µs ± 57.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f2850015310>
├── 'a' --> tensor([[ 0.4685,  0.0923,  1.8565],
│                   [ 0.4468,  1.4120, -1.1325],
│                   [ 0.4685,  0.0923,  1.8565],
│                   [ 0.4468,  1.4120, -1.1325],
│                   [ 0.4685,  0.0923,  1.8565],
│                   [ 0.4468,  1.4120, -1.1325],
│                   [ 0.4685,  0.0923,  1.8565],
│                   [ 0.4468,  1.4120, -1.1325],
│                   [ 0.4685,  0.0923,  1.8565],
│                   [ 0.4468,  1.4120, -1.1325],
│                   [ 0.4685,  0.0923,  1.8565],
│                   [ 0.4468,  1.4120, -1.1325],
│                   [ 0.4685,  0.0923,  1.8565],
│                   [ 0.4468,  1.4120, -1.1325],
│                   [ 0.4685,  0.0923,  1.8565],
│                   [ 0.4468,  1.4120, -1.1325]])
└── 'x' --> <FastTreeValue 0x7f27a2cd0f40>
    └── 'c' --> tensor([[ 0.3901, -1.8304,  1.2115,  0.6625],
                        [-0.3824, -0.3697, -1.0808, -1.1305],
                 

In [23]:
%timeit t_cat(trees)

30.2 µs ± 172 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

54.6 µs ± 55.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.4685,  0.0923,  1.8565],
                [ 0.4468,  1.4120, -1.1325]],
       
               [[ 0.4685,  0.0923,  1.8565],
                [ 0.4468,  1.4120, -1.1325]],
       
               [[ 0.4685,  0.0923,  1.8565],
                [ 0.4468,  1.4120, -1.1325]],
       
               [[ 0.4685,  0.0923,  1.8565],
                [ 0.4468,  1.4120, -1.1325]],
       
               [[ 0.4685,  0.0923,  1.8565],
                [ 0.4468,  1.4120, -1.1325]],
       
               [[ 0.4685,  0.0923,  1.8565],
                [ 0.4468,  1.4120, -1.1325]],
       
               [[ 0.4685,  0.0923,  1.8565],
                [ 0.4468,  1.4120, -1.1325]],
       
               [[ 0.4685,  0.0923,  1.8565],
                [ 0.4468,  1.4120, -1.1325]]]),
    x: Batch(
           c: tensor([[[ 0.3901, -1.8304,  1.2115,  0.6625],
                       [-0.3824, -0.3697, -1.0808, -1.1305],
                       [ 1.1624, -0.4730,  0.5559, -0.4295]],
         

In [26]:
%timeit Batch.stack(batches)

78.7 µs ± 382 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.4685,  0.0923,  1.8565],
               [ 0.4468,  1.4120, -1.1325],
               [ 0.4685,  0.0923,  1.8565],
               [ 0.4468,  1.4120, -1.1325],
               [ 0.4685,  0.0923,  1.8565],
               [ 0.4468,  1.4120, -1.1325],
               [ 0.4685,  0.0923,  1.8565],
               [ 0.4468,  1.4120, -1.1325],
               [ 0.4685,  0.0923,  1.8565],
               [ 0.4468,  1.4120, -1.1325],
               [ 0.4685,  0.0923,  1.8565],
               [ 0.4468,  1.4120, -1.1325],
               [ 0.4685,  0.0923,  1.8565],
               [ 0.4468,  1.4120, -1.1325],
               [ 0.4685,  0.0923,  1.8565],
               [ 0.4468,  1.4120, -1.1325]]),
    x: Batch(
           c: tensor([[ 0.3901, -1.8304,  1.2115,  0.6625],
                      [-0.3824, -0.3697, -1.0808, -1.1305],
                      [ 1.1624, -0.4730,  0.5559, -0.4295],
                      [ 0.3901, -1.8304,  1.2115,  0.6625],
                      [-0.3824, -

In [28]:
%timeit Batch.cat(batches)

143 µs ± 281 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

326 µs ± 1.51 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
